In [4]:
import random
import numpy as np

def get_distance_matrix(num_cities):
    distance_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            distance = float(input(f"Enter the distance between City {i + 1} and City {j + 1}: "))
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix

def initialize_population(num_cities, population_size):
    population = [list(range(num_cities)) for _ in range(population_size)]
    for population_path in population:
        random.shuffle(population_path)
    return population

def calculate_distance(population_path, distance_matrix):
    total_distance = 0
    for i in range(len(population_path) - 1):
        total_distance += distance_matrix[population_path[i]][population_path[i + 1]]
    total_distance += distance_matrix[population_path[-1]][population_path[0]]
    return total_distance

num_cities = int(input("Enter the number of cities: "))
distance_matrix = get_distance_matrix(num_cities)
population_size = 4
initial_population = initialize_population(num_cities, population_size)

print("Distance Matrix:")
for i in range(num_cities):
    for j in range(num_cities):
        print(f"{distance_matrix[i][j]:.2f}\t", end='')
    print()

distance_values = [calculate_distance(population_path, distance_matrix) for population_path in initial_population]
print("\nPopulation Paths and Distances:")
for i, (population_path, distance) in enumerate(zip(initial_population, distance_values)):
    print(f"Population Path {i + 1}: {population_path} Distance: {distance}")

Enter the number of cities: 5
Enter the distance between City 1 and City 2: 13
Enter the distance between City 1 and City 3: 25
Enter the distance between City 1 and City 4: 40
Enter the distance between City 1 and City 5: 10
Enter the distance between City 2 and City 3: 43
Enter the distance between City 2 and City 4: 75
Enter the distance between City 2 and City 5: 85
Enter the distance between City 3 and City 4: 46
Enter the distance between City 3 and City 5: 86
Enter the distance between City 4 and City 5: 18
Distance Matrix:
0.00	13.00	25.00	40.00	10.00	
13.00	0.00	43.00	75.00	85.00	
25.00	43.00	0.00	46.00	86.00	
40.00	75.00	46.00	0.00	18.00	
10.00	85.00	86.00	18.00	0.00	

Population Paths and Distances:
Population Path 1: [3, 1, 2, 0, 4] Distance: 171.0
Population Path 2: [3, 2, 4, 0, 1] Distance: 230.0
Population Path 3: [4, 2, 1, 0, 3] Distance: 200.0
Population Path 4: [4, 3, 0, 2, 1] Distance: 211.0


In [5]:
def apply_threshold_and_replace(distance_values, threshold):
    best_distance = min(distance_values)
    for i in range(len(distance_values)):
        if distance_values[i] > threshold:
            distance_values[i] = best_distance
    return distance_values

threshold = float(input("Enter the threshold: "))
distance_values = apply_threshold_and_replace(distance_values, threshold)
print("\nPopulation Paths and Distances After Applying Threshold:")
for i, (population_path, distance) in enumerate(zip(initial_population, distance_values)):
    print(f"Population Path {i + 1}: {population_path} Distance: {distance}")

Enter the threshold: 250

Population Paths and Distances After Applying Threshold:
Population Path 1: [3, 1, 2, 0, 4] Distance: 171.0
Population Path 2: [3, 2, 4, 0, 1] Distance: 230.0
Population Path 3: [4, 2, 1, 0, 3] Distance: 200.0
Population Path 4: [4, 3, 0, 2, 1] Distance: 211.0


In [6]:
def cyclic_crossover(parent1, parent2):
    size = len(parent1)
    offspring1 = [-1] * size
    offspring2 = [-1] * size
    start = 0

    while -1 in offspring1:
        index = start
        while offspring2[index] == -1:
            offspring1[index] = parent1[index]
            offspring2[index] = parent2[index]
            index = parent1.index(parent2[index])
        start += 1
    return offspring1, offspring2

In [7]:
# Inversion mutation function
def inversion_mutation(individual):
    i, j = random.sample(range(len(individual)), 2)
    if i > j:
        i, j = j, i
    individual[i:j] = reversed(individual[i:j])
    return individual

In [8]:
def genetic_algorithm(num_cities, distance_matrix, population_size, generations):
    population = initialize_population(num_cities, population_size)
    for generation in range(generations):
        distance_values = [calculate_distance(population_path, distance_matrix) for population_path in population]
        sorted_population = [population[i] for i in np.argsort(distance_values)]

        best_distance = min(distance_values)
        threshold = best_distance * 1.2
        distance_values = [distance if distance <= threshold else best_distance for distance in distance_values]

        offspring = []
        for i in range(population_size // 2):
            parent1 = sorted_population[i * 2]
            parent2 = sorted_population[i * 2 + 1]
            offspring1, offspring2 = cyclic_crossover(parent1, parent2)
            offspring.extend([offspring1, offspring2])

        print("\nOffspring After Crossover:")
        for i, individual in enumerate(offspring):
            print(f"Offspring {i + 1}: {individual} Distance: {calculate_distance(individual, distance_matrix)}")

        mutation_rate = 0.5
        mutated_population = [inversion_mutation(individual) for individual in offspring]

        print("\nMutation Results:")
        for i, individual in enumerate(mutated_population):
            print(f"Mutated Path {i + 1}: {individual} Distance: {calculate_distance(individual, distance_matrix)}")

        population = sorted_population[:population_size // 2] + mutated_population

    best_solution_index = np.argmin(distance_values)
    return population[best_solution_index]

generations = 2
best_solution = genetic_algorithm(num_cities, distance_matrix, population_size, generations)
print("\nBest Solution:")
print(f"Path: {best_solution} Distance: {calculate_distance(best_solution, distance_matrix)}")


Offspring After Crossover:
Offspring 1: [1, 3, 4, 0, 2] Distance: 171.0
Offspring 2: [0, 4, 3, 1, 2] Distance: 171.0
Offspring 3: [1, 3, 2, 0, 4] Distance: 241.0
Offspring 4: [3, 0, 1, 4, 2] Distance: 270.0

Mutation Results:
Mutated Path 1: [1, 0, 4, 3, 2] Distance: 130.0
Mutated Path 2: [0, 4, 3, 1, 2] Distance: 171.0
Mutated Path 3: [1, 3, 2, 0, 4] Distance: 241.0
Mutated Path 4: [3, 0, 4, 1, 2] Distance: 224.0

Offspring After Crossover:
Offspring 1: [1, 0, 4, 3, 2] Distance: 130.0
Offspring 2: [1, 3, 4, 0, 2] Distance: 171.0
Offspring 3: [0, 4, 3, 1, 2] Distance: 171.0
Offspring 4: [0, 4, 3, 1, 2] Distance: 171.0

Mutation Results:
Mutated Path 1: [1, 3, 4, 0, 2] Distance: 171.0
Mutated Path 2: [1, 0, 4, 3, 2] Distance: 130.0
Mutated Path 3: [1, 3, 4, 0, 2] Distance: 171.0
Mutated Path 4: [0, 4, 3, 1, 2] Distance: 171.0

Best Solution:
Path: [1, 0, 4, 3, 2] Distance: 130.0
